In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import display
from io import StringIO

URL = "https://lol.fandom.com/wiki/2025_Mid-Season_Invitational/Champion_Statistics"
API_PARSE_URL = (
    "https://lol.fandom.com/api.php?action=parse&prop=text&formatversion=2&format=json&"
    "page=2025_Mid-Season_Invitational/Champion_Statistics"
)
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://lol.fandom.com/",
}


def safe_read_html(html: str) -> list[pd.DataFrame]:
    try:
        return pd.read_html(StringIO(html))
    except ValueError:
        return []


def pick_champion_stats_table(tables: list[pd.DataFrame]) -> pd.DataFrame | None:
    """Pick the most likely Champion Statistics table from a list of DataFrames.

    Preference: table with a 'Champion' column (substring match) and many stat columns; tie-breaker by row count.
    """
    candidate_tables: list[tuple[int, pd.DataFrame]] = []
    for idx, df in enumerate(tables):
        columns = [str(c).strip() for c in df.columns]
        has_champion_col = any("champion" in col.lower() for col in columns)
        has_many_cols = len(columns) >= 8
        if has_champion_col and has_many_cols:
            candidate_tables.append((idx, df))
    if not candidate_tables:
        return None
    candidate_tables.sort(key=lambda x: x[1].shape[0], reverse=True)
    return candidate_tables[0][1]


# Source 1: normal page HTML
resp = requests.get(URL, headers=HEADERS, timeout=30)
resp.raise_for_status()
html_text = resp.text
all_tables_1 = safe_read_html(html_text)

# Source 2: action=render (server-rendered content only)
resp2 = requests.get(URL + "?action=render", headers=HEADERS, timeout=30)
resp2.raise_for_status()
render_html = resp2.text
all_tables_2 = safe_read_html(render_html)

# Source 3: MediaWiki parse API
resp3 = requests.get(API_PARSE_URL, headers=HEADERS, timeout=30)
resp3.raise_for_status()
api_json = resp3.json()
api_html = api_json.get("parse", {}).get("text", "")
all_tables_3 = safe_read_html(api_html)

# Aggregate and pick
all_sources = [("page", all_tables_1), ("render", all_tables_2), ("api", all_tables_3)]
all_tables = [t for _, tl in all_sources for t in tl]

table = pick_champion_stats_table(all_tables)

# Fallback: search by caption text using BeautifulSoup on the richest source
if table is None:
    soup = BeautifulSoup(render_html or html_text, "lxml")
    selected_html = None
    for tb in soup.find_all("table"):
        caption = tb.find("caption")
        caption_text = caption.get_text(strip=True) if caption else ""
        if "Champion Statistics" in caption_text:
            selected_html = str(tb)
            break
    if selected_html:
        tbls = safe_read_html(selected_html)
        if tbls:
            table = pick_champion_stats_table(tbls) or tbls[0]

if table is None:
    raise RuntimeError(
        f"Champion statistics table not found. Parsed counts — page:{len(all_tables_1)} render:{len(all_tables_2)} api:{len(all_tables_3)}"
    )

# Normalize columns (handle MultiIndex)
if isinstance(table.columns, pd.MultiIndex):
    table.columns = [
        " ".join([str(part) for part in col if pd.notna(part)]).strip() for col in table.columns.values
    ]
else:
    table.columns = [str(c).strip() for c in table.columns]

# Rename obvious variants to consistent names when present
rename_map = {"PB%": "Presence%", "WR": "WinRate", "KDA": "KDA", "CS/M": "CS_per_Min"}
for old, new in rename_map.items():
    if old in table.columns and new not in table.columns:
        table = table.rename(columns={old: new})

# Drop duplicate header rows and footer summary rows if any
champ_col_candidates = [c for c in table.columns if "champion" in str(c).lower()]
if champ_col_candidates:
    champ_col = champ_col_candidates[0]
    mask_is_header_row = table[champ_col].astype(str).str.lower().eq("champion")
    mask_is_total_row = table[champ_col].astype(str).str.contains(r"^total|totals$", case=False, na=False)
    table = table[~(mask_is_header_row | mask_is_total_row)].reset_index(drop=True)

# Best-effort type conversions on numeric-looking columns
for col in table.columns:
    if any(k in str(col).lower() for k in ["champ", "icon", "image"]):
        continue
    cleaned = (
        table[col]
        .astype(str)
        .str.replace("%", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.replace("—", "", regex=False)
        .str.replace("–", "", regex=False)
        .str.strip()
    )
    table[col] = pd.to_numeric(cleaned, errors="ignore")

display(table.head(20))


,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.1,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.2,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.3,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.4,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.5,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.6,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.7,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.8,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.9,...,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.13,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.14,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.15,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.16,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.17,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.18,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.19,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.20,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query 

In [15]:

# Create a new HTML table header with the specified structure
html_header = """<tr>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Champion</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><abbr title="Games (Contests)">G</abbr></th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">PB%</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">B</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><abbr title="Games Played">G</abbr></th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><abbr title="Played By Number of Players">By</abbr></th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">W</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">L</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">WR</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">K</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">D</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">A</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">KDA</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">CS</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">CS/M</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><abbr title="Gold">G</abbr></th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">G/M</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><abbr title="Damage">DMG</abbr></th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">DMG/M</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><abbr title="Kill Participation ((K+A)/Team K)">KPAR</abbr></th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><abbr title="Kill Share (K/Team K)">KS</abbr></th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><abbr title="Gold Share">GS</abbr></th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">As</th>
</tr>"""

# Display the HTML header
from IPython.display import HTML
display(HTML(f"<table>{html_header}</table>"))

# Keep the original table display for reference
display(table.head(20))



Champion,G,PB%,B,G,By,W,L,WR,K,D,A,KDA,CS,CS/M,G,G/M,DMG,DMG/M,KPAR,KS,GS,As


,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.1,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.2,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.3,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.4,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.5,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.6,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.7,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.8,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.9,...,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.13,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.14,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.15,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.16,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.17,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.18,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.19,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query Champion Statistics - 80 Total Games Played with 109 Champions Contested Restrict By Role (Query Permalinks):TopJungleMidBotSupport.20,Tournament: 2025 Mid-Season Invitational - Showing Values Per Game - Open As Query 

In [16]:
# Extract the column names from the HTML header
import re

# Extract column names from the HTML header
html_column_names = []
pattern = r'<th[^>]*>(?:<abbr title="([^"]+)">[^<]+</abbr>|([^<]+))</th>'
matches = re.findall(pattern, html_header)

for match in matches:
    # Each match is a tuple with two groups, one of which is empty
    title = match[0] if match[0] else match[1]
    html_column_names.append(title)

# Clean up the column names
html_column_names = [name.strip() for name in html_column_names]

# Rename the table columns to match the HTML header
if len(html_column_names) == len(table.columns):
    table.columns = html_column_names
else:
    print(f"Warning: Number of columns in HTML header ({len(html_column_names)}) doesn't match table columns ({len(table.columns)})")
    print("HTML columns:", html_column_names)
    print("Table columns:", table.columns.tolist())

# Display the table with the new column names
display(table.head(20))


,Champion,Games (Contests),PB%,B,Games Played,Played By Number of Players,W,L,WR,K,...,CS,CS/M,Gold,G/M,Damage,DMG/M,Kill Participation ((K+A)/Team K),Kill Share (K/Team K),Gold Share,As
0,Restrict By Role (Query Permalinks):,Top,Jungle,Mid,Bot,Support,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Varus,55,68.8%,41,14,7,9,5,64.3%,5.21,...,314.21,9.26,14.9,440,29.8k,878.3,70.5%,30.3%,23.4%,NaN
2,Twisted Fate,53,66.3%,45,8,5,0,8,0%,1.38,...,257.88,8.19,13,413,14.8k,469.8,66.1%,19.6%,24%,NaN
3,Pantheon,46,57.5%,33,13,9,9,4,69.2%,5.08,...,233.15,6.75,13.7,397,20.6k,595,69.6%,26.7%,20.7%,NaN
4,Azir,45,56.3%,35,10,7,6,4,60%,4.5,...,307,9.17,14.6,435,26.8k,800,69.9%,33.1%,23.5%,NaN
5,Vi,41,51.2%,24,17,9,7,10,41.2%,1.94,...,209.53,6.42,11.4,350,10.5k,322.2,63.2%,13.6%,19.1%,NaN
6,Taliyah,41,51.2%,26,15,9,10,5,66.7%,3.87,...,272.6,8.38,13.3,409,22.9k,704,74.3%,22.9%,21.7%,NaN
7,Gwen,38,47.5%,29,9,5,4,5,44.4%,3.56,...,247.11,7.65,13,403,20.6k,637.7,61.9%,25.4%,22%,NaN
8,Poppy,35,43.8%,19,16,11,9,7,56.3%,2.38,...,153.38,4.77,10.5,328,12k,373.8,71.9%,14.6%,17.4%,NaN
9,Rumble,34,42.5%,17,17,8,8,9,47.1%,3.88,...,256.47,7.57,13.1,386,25k,737.3,64.2%,22.3%,20.8%,NaN


In [17]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Champion                           110 non-null    object
 1   Games (Contests)                   110 non-null    object
 2   PB%                                110 non-null    object
 3   B                                  110 non-null    object
 4   Games Played                       110 non-null    object
 5   Played By Number of Players        110 non-null    object
 6   W                                  109 non-null    object
 7   L                                  109 non-null    object
 8   WR                                 109 non-null    object
 9   K                                  109 non-null    object
 10  D                                  109 non-null    object
 11  A                                  109 non-null    object
 12  KDA     

In [19]:
# Drop the last column ('As' column)
table = table.drop(columns=['As'])

# Drop the first row (index 0 which contains "Restrict By Role (Query Permalinks):")
table = table.drop(index=0)

# Display the modified table
print("Table after dropping the last column and first row:")
display(table.head(20))


Table after dropping the last column and first row:


,Champion,Games (Contests),PB%,B,Games Played,Played By Number of Players,W,L,WR,K,...,KDA,CS,CS/M,Gold,G/M,Damage,DMG/M,Kill Participation ((K+A)/Team K),Kill Share (K/Team K),Gold Share
1,Varus,55,68.8%,41,14,7,9,5,64.3%,5.21,...,4.36,314.21,9.26,14.9,440,29.8k,878.3,70.5%,30.3%,23.4%
2,Twisted Fate,53,66.3%,45,8,5,0,8,0%,1.38,...,1.28,257.88,8.19,13,413,14.8k,469.8,66.1%,19.6%,24%
3,Pantheon,46,57.5%,33,13,9,9,4,69.2%,5.08,...,5.38,233.15,6.75,13.7,397,20.6k,595,69.6%,26.7%,20.7%
4,Azir,45,56.3%,35,10,7,6,4,60%,4.5,...,4.13,307,9.17,14.6,435,26.8k,800,69.9%,33.1%,23.5%
5,Vi,41,51.2%,24,17,9,7,10,41.2%,1.94,...,2.59,209.53,6.42,11.4,350,10.5k,322.2,63.2%,13.6%,19.1%
6,Taliyah,41,51.2%,26,15,9,10,5,66.7%,3.87,...,4.7,272.6,8.38,13.3,409,22.9k,704,74.3%,22.9%,21.7%
7,Gwen,38,47.5%,29,9,5,4,5,44.4%,3.56,...,2.44,247.11,7.65,13,403,20.6k,637.7,61.9%,25.4%,22%
8,Poppy,35,43.8%,19,16,11,9,7,56.3%,2.38,...,4.56,153.38,4.77,10.5,328,12k,373.8,71.9%,14.6%,17.4%
9,Rumble,34,42.5%,17,17,8,8,9,47.1%,3.88,...,3.58,256.47,7.57,13.1,386,25k,737.3,64.2%,22.3%,20.8%
10,Lucian,32,40%,19,13,8,8,5,61.5%,6.46,...,5.13,296.54,9.19,14.8,457,23.9k,740,66.8%,35.3%,24%


In [20]:
# Export the cleaned table to a CSV file
csv_filename = "msi_2025_champion_stats.csv"
table.to_csv(csv_filename, index=False)
print(f"Table exported to {csv_filename}")


Table exported to msi_2025_champion_stats.csv
